<a href="https://colab.research.google.com/github/harshinirhsv/TOPIC-MODELING-on-research-articles/blob/main/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TOPIC MODELING ON RESEARCH ARTICLES

PROBLEM STATEMENT:
Topic Modeling for Research Articles Researchers have access to large online archives of scientific articles. As a consequence, finding relevant articles has become more difficult. Tagging or topic modelling provides a way to give token of identification to research articles which facilitates recommendation and search process.

Given the abstract and title for a set of research articles, predict the topics for each article included in the test set.

Note that a research article can possibly have more than 1 topic. The research article abstracts and titles are sourced from the following 6 topics:

Computer Science
Physics
Mathematics
Statistics
Quantitative Biology
Quantitative Finance

In [96]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [97]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [98]:
data = pd.read_csv("train.csv",encoding = "ISO-8859-1")
df_test = pd.read_csv("test.csv")
data_original = data.copy() 
df_test_original = df_test.copy()

In [99]:
data.head()
#df_test.head()
#df_test.drop(['ID'], axis=1)
df_test.shape

(8989, 3)

In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20972 entries, 0 to 20971
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    20972 non-null  int64 
 1   TITLE                 20972 non-null  object
 2   ABSTRACT              20972 non-null  object
 3   Computer Science      20972 non-null  int64 
 4   Physics               20972 non-null  int64 
 5   Mathematics           20972 non-null  int64 
 6   Statistics            20972 non-null  int64 
 7   Quantitative Biology  20972 non-null  int64 
 8   Quantitative Finance  20972 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 1.4+ MB


In [101]:
df_test.isnull()

,ID,TITLE,ABSTRACT
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
8984,False,False,False
8985,False,False,False
8986,False,False,False
8987,False,False,False


In [102]:
data.describe()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
count,20972.000000,20972.000000,20972.000000,20972.000000,20972.000000,20972.000000,20972.000000
mean,10486.500000,0.409784,0.286716,0.267881,0.248236,0.027990,0.011873
std,6054.239259,0.491806,0.452238,0.442866,0.432000,0.164947,0.108317
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5243.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10486.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15729.250000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,20972.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [103]:
abstract = pd.DataFrame(data['ABSTRACT'])
abstract

,ABSTRACT
0,"Predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. Given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. While global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands. In this article, we..."
1,"Rotation invariance and translation invariance have great values in image\nrecognition tasks. In this paper, we bring a new architecture in convolutional\nneural network (CNN) named cyclic convolutional layer to achieve rotation\ninvariance in 2-D symbol recognition. We can also get the position and\norientation of the 2-D symbol by the network to achieve detection purpose for\nmultiple non-overlap target. Last but not least, this architecture can achieve\none-shot learning in some cases using those invariance.\n"
2,"We introduce and develop the notion of spherical polyharmonics, which are a\nnatural generalisation of spherical harmonics. In particular we study the\ntheory of zonal polyharmonics, which allows us, analogously to zonal harmonics,\nto construct Poisson kernels for polyharmonic functions on the union of rotated\nballs. We find the representation of Poisson kernels and zonal polyharmonics in\nterms of the Gegenbauer polynomials. We show the connection between the\nclassical Poisson kernel for harmonic functions on the ball, Poisson kernels\nfor polyharmonic functions on the union of rotat..."
3,"The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the\nMaxwell equations (the so called stochastic MLLG system) describes the creation\nof domain walls and vortices (fundamental objects for the novel nanostructured\nmagnetic memories). We first reformulate the stochastic LLG equation into an\nequation with time-differentiable solutions. We then propose a convergent\n$\theta$-linear scheme to approximate the solutions of the reformulated system.\nAs a consequence, we prove convergence of the approximate solutions, with no or\nminor conditions on time and space steps (de..."
4,"Fourier-transform infra-red (FTIR) spectra of samples from 7 plant species\nwere used to explore the influence of preprocessing and feature extraction on\nefficiency of machine learning algorithms. Wavelet Tensor Train (WTT) and\nDiscrete Wavelet Transforms (DWT) were compared as feature extraction\ntechniques for FTIR data of medicinal plants. Various combinations of signal\nprocessing steps showed different behavior when applied to classification and\nclustering tasks. Best results for WTT and DWT found through grid search were\nsimilar, significantly improving quality of clustering as..."
...,...
20967,"Machine learning is finding increasingly broad application in the physical\nsciences. This most often involves building a model relationship between a\ndependent, measurable output and an associated set of controllable, but\ncomplicated, independent inputs. We present a tutorial on current techniques in\nmachine learning -- a jumping-off point for interested researchers to advance\ntheir work. We focus on deep neural networks with an emphasis on demystifying\ndeep learning. We begin with background ideas in machine learning and some\nexample applications from current research in plasma p..."
20968,"Polycrystalline diamond coatings have been grown on cemented carbide\nsubstrates with different aspect ratios by a microwave plasma CVD in\nmethane-hydrogen gas mixtures. To protect the edges of the substrates from\nnon-uniform heating due to the plasma edge effect, a special plateholder with\npockets for group growth has been used. The difference in heights of the\nsubstrates and plateholder, and its influence on the diamond film mea

In [104]:
books = pd.DataFrame(data['TITLE'])
books

,TITLE
0,Reconstructing Subject-Specific Effect Maps
1,Rotation Invariance Neural Network
2,Spherical polyharmonics and Poisson kernels for polyharmonic functions
3,A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system
4,Comparative study of Discrete Wavelet Transforms and Wavelet Tensor Train decomposition to feature extraction of FTIR data of medicinal plants
...,...
20967,Contemporary machine learning: a guide for practitioners in the physical sciences
20968,Uniform diamond coatings on WC-Co hard alloy cutting inserts deposited by a microwave plasma CVD
20969,Analysing Soccer Games with Clustering and Conceptors
20970,On the Efficient Simulation of the Left-Tail of the Sum of Correlated Log-normal Variates


In [105]:
data_genre = data.iloc[:,3:9]
genre = pd.DataFrame(data_genre)
#genre
data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,"Predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. Given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. While global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands. In this article, we...",1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,"Rotation invariance and translation invariance have great values in image\nrecognition tasks. In this paper, we bring a new architecture in convolutional\nneural network (CNN) named cyclic convolutional layer to achieve rotation\ninvariance in 2-D symbol recognition. We can also get the position and\norientation of the 2-D symbol by the network to achieve detection purpose for\nmultiple non-overlap target. Last but not least, this architecture can achieve\none-shot learning in some cases using those invariance.\n",1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels for polyharmonic functions,"We introduce and develop the notion of spherical polyharmonics, which are a\nnatural generalisation of spherical harmonics. In particular we study the\ntheory of zonal polyharmonics, which allows us, analogously to zonal harmonics,\nto construct Poisson kernels for polyharmonic functions on the union of rotated\nballs. We find the representation of Poisson kernels and zonal polyharmonics in\nterms of the Gegenbauer polynomials. We show the connection between the\nclassical Poisson kernel for harmonic functions on the ball, Poisson kernels\nfor polyharmonic functions on the union of rotat...",0,0,1,0,0,0
3,4,A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system,"The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the\nMaxwell equations (the so called stochastic MLLG system) describes the creation\nof domain walls and vortices (fundamental objects for the novel nanostructured\nmagnetic memories). We first reformulate the stochastic LLG equation into an\nequation with time-differentiable solutions. We then propose a convergent\n$\theta$-linear scheme to approximate the solutions of the reformulated system.\nAs a consequence, we prove convergence of the approximate solutions, with no or\nminor conditions on time and space steps (de...",0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transforms and Wavelet Tensor Train decomposition to feature extraction of FTIR data of medicinal plants,"Fourier-transform infra-red (FTIR) spectra of samples from 7 plant species\nwere used to explore the influence of preprocessing and feature extraction on\nefficiency of machine learning algorithms. Wavelet Tensor Train (WTT) and\nDiscrete Wavelet Transforms (DWT) were compared as feature extraction\ntechniques for FTIR data of medicinal plants. Various combinations of signal\nprocessing steps showed different behavior when applied to classification and\nclustering tasks. Best results for WTT and DWT found through grid search were\nsimilar, significantly improving quality of clustering as...",1,0,0,1,0,0


In [106]:
data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,"Predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. Given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. While global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands. In this article, we...",1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,"Rotation invariance and translation invariance have great values in image\nrecognition tasks. In this paper, we bring a new architecture in convolutional\nneural network (CNN) named cyclic convolutional layer to achieve rotation\ninvariance in 2-D symbol recognition. We can also get the position and\norientation of the 2-D symbol by the network to achieve detection purpose for\nmultiple non-overlap target. Last but not least, this architecture can achieve\none-shot learning in some cases using those invariance.\n",1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels for polyharmonic functions,"We introduce and develop the notion of spherical polyharmonics, which are a\nnatural generalisation of spherical harmonics. In particular we study the\ntheory of zonal polyharmonics, which allows us, analogously to zonal harmonics,\nto construct Poisson kernels for polyharmonic functions on the union of rotated\nballs. We find the representation of Poisson kernels and zonal polyharmonics in\nterms of the Gegenbauer polynomials. We show the connection between the\nclassical Poisson kernel for harmonic functions on the ball, Poisson kernels\nfor polyharmonic functions on the union of rotat...",0,0,1,0,0,0
3,4,A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system,"The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the\nMaxwell equations (the so called stochastic MLLG system) describes the creation\nof domain walls and vortices (fundamental objects for the novel nanostructured\nmagnetic memories). We first reformulate the stochastic LLG equation into an\nequation with time-differentiable solutions. We then propose a convergent\n$\theta$-linear scheme to approximate the solutions of the reformulated system.\nAs a consequence, we prove convergence of the approximate solutions, with no or\nminor conditions on time and space steps (de...",0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transforms and Wavelet Tensor Train decomposition to feature extraction of FTIR data of medicinal plants,"Fourier-transform infra-red (FTIR) spectra of samples from 7 plant species\nwere used to explore the influence of preprocessing and feature extraction on\nefficiency of machine learning algorithms. Wavelet Tensor Train (WTT) and\nDiscrete Wavelet Transforms (DWT) were compared as feature extraction\ntechniques for FTIR data of medicinal plants. Various combinations of signal\nprocessing steps showed different behavior when applied to classification and\nclustering tasks. Best results for WTT and DWT found through grid search were\nsimilar, significantly improving quality of clustering as...",1,0,0,1,0,0


In [107]:
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(data[['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']])
df_std = std_scale.transform(data[['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']])

minmax_scale = preprocessing.MinMaxScaler().fit(data[['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']])
df_minmax = minmax_scale.transform(data[['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']])

In [108]:
from sklearn.preprocessing import LabelEncoder

feat = ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']
for x in feat:
    le = LabelEncoder()
    le.fit(list(genre[x].values))
    genre[x] = le.transform(list(genre[x]))

In [109]:
data['everything'] = pd.DataFrame(data['TITLE'] + ' ' + data['ABSTRACT'])
print (data['everything'].head(5))

0    Reconstructing Subject-Specific Effect Maps   Predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. Given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. While global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndis...
1                                                Rotation Invariance Neural Network   Rotation invariance and translation invariance have great values in image\nrecognition tasks. In this paper, we bring a new architecture in convolutional\nneural network (CNN) named cyclic convolutional layer to achieve rotation\ninvariance in 2-D symbol recognition. We can also get the position and\norienta

In [110]:
'''from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(data[['everything']])
df_std = std_scale.transform(data[['everything']])

minmax_scale = preprocessing.MinMaxScaler().fit(data[['everything']])
df_minmax = minmax_scale.transform(data[['everything']])'''

"from sklearn import preprocessing\n\nstd_scale = preprocessing.StandardScaler().fit(data[['everything']])\ndf_std = std_scale.transform(data[['everything']])\n\nminmax_scale = preprocessing.MinMaxScaler().fit(data[['everything']])\ndf_minmax = minmax_scale.transform(data[['everything']])"

In [112]:
def change(t):
    t = t.split()
    return ' '.join([(i) for (i) in t if i not in stop])

In [113]:
import nltk

In [115]:
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))

In [ ]:
remove_list = ['the','at','or']
for i,j in enumerate(data['everything']):
    t = data['everything'][i].split()
    data['everything'][i] = ' '.join([str(i) for (i) in t if i not in stop])

In [118]:
data['everything'].apply(change)

0        Reconstructing Subject-Specific Effect Maps Predictive models allow subject-specific inference analyzing disease related alterations neuroimaging data. Given subject's data, inference made two levels: global, i.e. identifiying condition presence subject, local, i.e. detecting condition effect individual measurement extracted subject's data. While global inference widely used, local inference, used form subject-specific effect maps, rarely used existing models often yield noisy detections composed dispersed isolated islands. In article, propose reconstruction method, named RSM, improve subj...
1                                                                                                                                                             Rotation Invariance Neural Network Rotation invariance translation invariance great values image recognition tasks. In paper, bring new architecture convolutional neural network (CNN) named cyclic convolutional layer achieve rotation

In [119]:
vectorizer = TfidfVectorizer(min_df=2, max_features=90000, strip_accents='unicode',lowercase =True,
                            analyzer='word', token_pattern=r'\w+', use_idf=True, 
                            smooth_idf=True, sublinear_tf=True, stop_words = 'english')
vectors = vectorizer.fit_transform(data['everything'])
vectors.shape

(20972, 28130)

In [120]:
'''print (x_train.shape)
print (y_train.shape)
print (df_test.shape)'''

'print (x_train.shape)\nprint (y_train.shape)\nprint (df_test.shape)'

In [121]:
type(books)

pandas.core.frame.DataFrame

In [122]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

In [123]:
X_train, X_test, y_train, y_test = train_test_split(vectors, genre, test_size=0.5)


In [124]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)
#print(X_train)
#print(y_train)
#print(X_test)
print(y_test)

(10486, 28130)
(10486, 6)
(10486, 28130)
(10486, 6)
       Computer Science  Physics  Mathematics  Statistics  \
6357                  0        1            0           0   
8887                  0        0            1           0   
7783                  1        0            0           0   
6934                  1        0            0           0   
12525                 1        0            0           0   
...                 ...      ...          ...         ...   
2921                  0        0            0           0   
13828                 0        0            1           0   
6677                  1        0            0           1   
4720                  0        0            0           1   
12443                 0        1            0           0   

       Quantitative Biology  Quantitative Finance  
6357                      0                     0  
8887                      0                     0  
7783                      0                     0  
6934   

In [125]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation='logistic', alpha=0.00003, batch_size='auto',
                   beta_1=0.9, beta_2=0.999, early_stopping=False,
                   epsilon=1e-08, hidden_layer_sizes=(20,), learning_rate='constant',
                   learning_rate_init=0.003, max_iter=400, momentum=0.9,
                   nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
                   solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
                   warm_start=False)
clf.fit(X_train, y_train) 
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

0.6744727807536833
0.6342742704558458


In [126]:
print(X_test)

  (0, 10052)	0.1862317572155156
  (0, 23810)	0.158058575046369
  (0, 9885)	0.34058967151713737
  (0, 8119)	0.16406029214985182
  (0, 8120)	0.27178779095011824
  (0, 27150)	0.16925339168610357
  (0, 18916)	0.16330464792022603
  (0, 2230)	0.15864857499092122
  (0, 2467)	0.1289466570980697
  (0, 1558)	0.1289466570980697
  (0, 13048)	0.1419989738761465
  (0, 15208)	0.15692496326863695
  (0, 24440)	0.1592551968204184
  (0, 5632)	0.11486523284976534
  (0, 20472)	0.10865082038460548
  (0, 17554)	0.11349996683927549
  (0, 16299)	0.110679739383176
  (0, 9952)	0.15211714000718757
  (0, 25467)	0.1400157761316061
  (0, 9498)	0.1219391247395853
  (0, 17664)	0.0815989527987507
  (0, 25964)	0.11357358623021943
  (0, 11102)	0.09269376778410156
  (0, 24584)	0.1428435390203326
  (0, 7203)	0.08708717462659163
  :	:
  (10485, 17545)	0.03647398491106371
  (10485, 10482)	0.04977552868361057
  (10485, 17704)	0.0697432030861252
  (10485, 23215)	0.07487637928084902
  (10485, 15982)	0.07596518670899324
  (10485

In [128]:
submission = pd.read_csv("sample_submission_UVKGLZE.csv")
#submission.head()
submission.shape

(8989, 7)

In [129]:
genre1 = pd.DataFrame(submission.iloc[:,1:])
genre1.head()
genre1.shape

(8989, 6)

In [130]:
submission1 = pd.DataFrame(submission.iloc[:,:1])
submission1.head()
submission.shape

(8989, 7)

In [131]:
print(pred.shape)
print(genre1.shape)

(10486, 6)
(8989, 6)


In [132]:
genre1 = pred
genre2 = pd.DataFrame(genre1)
print(genre2.head())
submission1 = df_test['ID']
submission2 = pd.DataFrame(submission1)
print(submission1.head())
#submission3 = pd.concat([submission2,genre2],)
submission3 = pd.concat([submission2,genre2], axis=1)
submission3.head()

   0  1  2  3  4  5
0  0  1  0  0  0  0
1  0  0  1  0  0  0
2  1  0  0  0  0  0
3  0  0  0  0  0  0
4  1  0  0  1  0  0
0    20973
1    20974
2    20975
3    20976
4    20977
Name: ID, dtype: int64


,ID,0,1,2,3,4,5
0,20973.0,0,1,0,0,0,0
1,20974.0,0,0,1,0,0,0
2,20975.0,1,0,0,0,0,0
3,20976.0,0,0,0,0,0,0
4,20977.0,1,0,0,1,0,0


In [137]:
submission3.rename(columns = {0:'Computer Science', 1:'Physics',2:'Statistics',3:'Mathematics',4:'Statistics',
                              5:'Quantitative Biology',6:'Quantitative Finance'}, inplace = True) 
#submission3.head()
submission3 = submission3.astype("Int64")
submission4 = submission3[:8989]
submission4

,ID,Computer Science,Physics,Statistics,Mathematics,Statistics,Quantitative Biology
0,20973,0,1,0,0,0,0
1,20974,0,0,1,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,0,0,0,0,0
4,20977,1,0,0,1,0,0
...,...,...,...,...,...,...,...
8984,29957,1,1,0,0,0,0
8985,29958,0,0,0,0,0,0
8986,29959,1,0,0,1,0,0
8987,29960,0,1,0,0,0,0


In [134]:
submission3.to_csv('Sample_submission.csv', header=False, index=False)

In [138]:
submission4.to_csv(r'Submissionfinal.csv', index=False) 